In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time


In [ ]:
# Load the overlay
overlay = Overlay('./radiation_overlay.bit')

# Access the radiation detector IP
radiation_detector = overlay.radiation_detector_top

# Register addresses
ADDR_THRESHOLD   = 0x00
ADDR_RANDOM_SEED = 0x04
ADDR_NOISE_AMP   = 0x08
ADDR_SPIKE_INT   = 0x0C
ADDR_SPIKE_AMP   = 0x10

# Configure simulation parameters
def configure_simulation(threshold=2048, seed=0x12345678, 
                       noise_amp=256, spike_interval=100000, 
                       spike_amp=4096):
    """Configure the radiation signal simulator parameters
    
    Args:
        threshold (int): Detection threshold level (0-4095)
        seed (int): Random seed for noise generation
        noise_amp (int): Amplitude of background noise (0-4095)
        spike_interval (int): Clock cycles between radiation spikes
        spike_amp (int): Amplitude of radiation spikes (0-4095)
    """
    radiation_detector.write(ADDR_THRESHOLD, threshold)
    radiation_detector.write(ADDR_RANDOM_SEED, seed)
    radiation_detector.write(ADDR_NOISE_AMP, noise_amp)
    radiation_detector.write(ADDR_SPIKE_INT, spike_interval)
    radiation_detector.write(ADDR_SPIKE_AMP, spike_amp)
    
# Initialize with default values
configure_simulation()


In [ ]:
# Configure DMA buffer for data transfer
BUFFER_SIZE = 1024
input_buffer = allocate(shape=(BUFFER_SIZE,), dtype=np.uint32)
output_buffer = allocate(shape=(BUFFER_SIZE,), dtype=np.uint32)

# Configure detector parameters
def configure_detector(threshold=2048):
    radiation_detector.write(0x10, threshold)  # Set threshold
    radiation_detector.write(0x14, 1)         # Enable
    
# Read detector status
def read_status():
    status = radiation_detector.read(0x00)
    event_count = radiation_detector.read(0x04)
    return {
        'alert': bool(status & 0x1),
        'event_count': event_count
    }


In [ ]:
# Real-time monitoring function
def monitor_radiation(update_interval=1.0, max_time=60):
    start_time = time.time()
    times = []
    counts = []
    
    plt.figure(figsize=(12, 6))
    while time.time() - start_time < max_time:
        status = read_status()
        current_time = time.time() - start_time
        
        times.append(current_time)
        counts.append(status['event_count'])
        
        clear_output(wait=True)
        plt.plot(times, counts, 'b-')
        plt.xlabel('Time (s)')
        plt.ylabel('Event Count')
        plt.title('Radiation Events Over Time')
        plt.grid(True)
        plt.show()
        
        if status['alert']:
            print(f"⚠️ Alert! High radiation level detected at t={current_time:.1f}s")
            
        time.sleep(update_interval)
        
    return np.array(times), np.array(counts)


In [ ]:
# Example: Configure different simulation scenarios
def configure_normal_background():
    """Configure normal background radiation simulation"""
    configure_simulation(
        threshold=2048,
        noise_amp=128,
        spike_interval=1000000,  # Very rare spikes
        spike_amp=2048  # Moderate spikes
    )

def configure_high_activity():
    """Configure simulation with frequent radiation events"""
    configure_simulation(
        threshold=2048,
        noise_amp=256,
        spike_interval=50000,   # Frequent spikes
        spike_amp=4096  # High amplitude spikes
    )

def configure_equipment_malfunction():
    """Simulate a detector malfunction with erratic readings"""
    configure_simulation(
        threshold=2048,
        noise_amp=1024,  # Very noisy
        spike_interval=10000,  # Very frequent spikes
        spike_amp=3072  # Moderate-high spikes
    )
